# Rede Neural

In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# Visão Computacional

In [2]:
import cv2 
import numpy as np
import time

import PC as pc

baixo = [2, 0, 0]
alto  = [30, 255, 255]

info = []
idx = 1

sensibilidade = []
mediaHistograma = []

print("Executando...\n\n")

cap = cv2.VideoCapture(0)

while(True):
            
    ret, frame = cap.read()
    
    image_np = np.array(frame)
    
    areaMangaTeste = np.array(frame)
    
    category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')
        
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor) 

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    imagem_pedunculo = image_np.copy()
    
    detection_threshold = 0.7

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                instance_masks=detections.get('detection_masks'),
                use_normalized_coordinates=True,
                max_boxes_to_draw=1,
                min_score_thresh=.8,
                skip_scores= False,
                skip_labels= True,
                #skip_boxes= False,
                agnostic_mode=True)
  
    scores = list(filter(lambda x: x> detection_threshold, 
    detections['detection_scores']))
    boxes = detections['detection_boxes'][:len(scores)]
    classes = detections['detection_classes'][:len(scores)]
    
    width = image_np_with_detections.shape[1]
    height = image_np_with_detections.shape[0]
    
    sensibilidade.append(scores)
    
    try:
        
        if(len(scores) > 0):

            for idxb, box in enumerate(boxes):

                roi = box * [height, width, height, width]

                xt = roi[1] #- 120     #linha esquerda
                xb = roi[3]           #linha direita
                yt = roi[0] #- 130     #linha superior
                yb = roi[2]           #linha inferior

                #xt = xt - (xt * 0.1)
                #xb = xb + (xb * 0.05)
                #yt = yt - (yt * 0.1)
                #yb = yb + (yb * 0.05)

                tamanhoFruta = (yt + yb) / 2
                centroFrutaX = (xt + xb) / 2

                centroFrutaX = int(centroFrutaX)
                tamanhoFruta = int(tamanhoFruta)

                xt = int(xt)
                xb = int(xb)
                yt = int(yt)
                yb = int(yb)

                areaManga = image_np_with_detections[yt:yb, xt:xb]

                #cv2.rectangle(image_np_with_detections, (xt,yt), (xb,yb), (255,0,0), 2)

                #Chama a função que prevê a área do pedúnculo
                areaPedunculo = pc.preveAreaPedunculo(xt, yt, xb, yb, 0.3, 0.5, -10)

                x1, y1, x2, y2 = areaPedunculo[0], areaPedunculo[1], areaPedunculo[2], areaPedunculo[3]

                #Marca um quadrado na região do pedúnculo        
                cv2.rectangle(image_np_with_detections, (x1,y1), (x2,y2), (50,0,255), 3)

                TopLeftX = x1
                TopLeftY = y2

                #Faz o corte na área prevista do pedúnculo
                corteAreaPedunculo = imagem_pedunculo[y2:y1, x1:x2]  

                #Chama a função para encontrar o ponto de corte        
                coordenadaFinal = pc.coordenadaPontoFinal(corteAreaPedunculo,
                                                              baixo,
                                                              alto,
                                                              TopLeftX, 
                                                              TopLeftY, 
                                                              "Ponderada", 
                                                              7,
                                                             2)

                pontoX, pontoY, areaSegmentada, pontosCandidatos, valorHue = coordenadaFinal[0], coordenadaFinal[1], coordenadaFinal[2], coordenadaFinal[3], coordenadaFinal[4]

                mediaHistograma.append(valorHue)

                cv2.circle(image_np_with_detections, (pontoX, pontoY), 10, (0,0,255), -1)

                cv2.line(image_np_with_detections, (pontoX, pontoY), (int((xt+xb)/2), int(yt)), (0,255,0), 3)

                cv2.circle(image_np_with_detections, (int((xt+xb)/2), int(yt)), 10, (255,0,90), -1)

                cv2.imshow('Area Segmentada', cv2.resize(areaManga, (200, 200)))

    except:

        cv2.putText(image_np_with_detections, "Manga muito proxima", (25, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
    
    cv2.imshow("Tempo Real", image_np_with_detections)
    
    
    #cv2.imshow('Area Segmentada', cv2.resize(areaSegmentada, (200, 200)))
    #cv2.imshow('Pontos canditatos', cv2.resize(pontosCandidatos, (200, 200)))
    
    if cv2.waitKey(1) == 27:

        #cv2.imwrite("RESULTADOS/TEMPO_REAL/" + "1_area_pedunculo.jpg", corteAreaPedunculo)        
        #cv2.imwrite("RESULTADOS/TEMPO_REAL/" + "2_area_segmentada.jpg", areaSegmentada)        
        #cv2.imwrite("RESULTADOS/TEMPO_REAL/" + "3_ponto_cadidatos.jpg", pontosCandidatos)
        #cv2.imwrite("RESULTADOS/TEMPO_REAL/" + "4_area_manga.png", areaManga)
        #cv2.imwrite("RESULTADOS/TEMPO_REAL/" + "5_final.jpg", image_np_with_detections)

        break
            

cap.release()
cv2.destroyAllWindows()


print("\n\nFinalizando...")

Executando...




Finalizando...
